In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

from tqdm import tqdm


#sns.set(color_codes=True)

In [2]:
# Make proper file names for Windows
# pip install regex

import re

def windows_filename(s):
    s = str(s).strip().replace(' ', '_')
    return re.sub(r'(?u)[^-\w.]', '', s)

In [3]:
def plot_graph(plt,start_time, variable):
    plt.savefig(windows_filename(start_time)+'_' + variable+ '.pdf')    

In [49]:
def ftoc(f):
    return (f - 32) * 5.0/9.0

def mphtoms(mph):
    return mph * 0.44704

In [45]:
from pathlib import Path

basePathPatrick = Path("C:/Users/pkastner/Google Drive/Patrick/OutdoorComfort-Biking/Measurements/")
basePathESL = Path("C:/Users/pkastner/Google Drive/Patrick/OutdoorComfort-Biking/Measurements/Remy/_Stations/")
basePathRemy = Path("C:/Users/remym/Dropbox/Cornell 2019-2020/ESL Independent Study/_Stations")
basePathRemy = Path("C:/Users/remym/Dropbox/Cornell 2019-2020/ESL Independent Study/_StationDataPython")
basePathBenj = Path("C:/Users/pkastner/Google Drive/Patrick/OutdoorComfort-Biking/Measurements/Benjamin_StationFile")

loadPath = basePathPatrick

In [ ]:
from hoboreader import HoboReader
S1=HoboReader(loadPath / "20191101_11-04_DL1.csv") 
S2=HoboReader(loadPath / "20191101-10-50_DL2.csv")
S3=HoboReader(loadPath / "20191115-11-08_DL3.csv")
S4=HoboReader(loadPath / "20191101-12-58_DL4.csv")


df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()

# Variable to plot
variable1 = "Wind Speed"
variable2 = "Wind Direction"

#df1[variable].hist()
sns.distplot(df1[variable1], kde=False, fit=stats.gamma);
sns.distplot(df2[variable1], kde=False, fit=stats.gamma);
sns.distplot(df3[variable1], kde=False, fit=stats.gamma);
sns.distplot(df4[variable1], kde=False, fit=stats.gamma);

#sns.jointplot(x=variable1, y=variable2, data=df1);

In [ ]:
from windrose import WindroseAxes
from windrose import WindAxes
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import numpy as np

def plotWR(ws, wd):
    ax = WindroseAxes.from_ax()
    ax.contourf(wd, ws, bins=np.arange(0, 6, 1),nsector= 36, cmap=cm.hot)
    ax.contour(wd, ws, bins=np.arange(0, 6, 1),nsector= 36, colors='black', lw=1)
    ax.set_legend()
    
def printPDF(ws):
    ax = WindAxes.from_ax()
    bins = np.arange(0, 6 + 1, 0.5)
    bins = bins[1:]
    ax, params = ax.pdf(ws, bins=bins)

In [ ]:


list = [df1, df2, df3, df4]

# Variable to plot
variable1 = "Wind Speed"
variable2 = "Wind Direction"

ws = np.array(df1[variable1]).flatten()
wd = np.array(df1[variable2]).flatten()

for i in tqdm(list):
    ws = np.array(i[variable1]).flatten()
    wd = np.array(i[variable2]).flatten()
    plotWR(ws, wd)
    printPDF(ws)
    
     
    


In [ ]:
from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path

      
    
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   
  
    
S1=HoboReader(loadPath / "20191101_11-04_DL1.csv") 
S2=HoboReader(loadPath / "20191101-10-50_DL2.csv")
S3=HoboReader(loadPath / "20191115-11-08_DL3.csv")
S4=HoboReader(loadPath / "20191101-12-58_DL4.csv")


df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()

# Select time

start_time = '2019-10-17 06:00:00-04:00'
end_time = '2019-10-17 19:00:00-04:00'

df1_ = df1.loc[start_time:end_time]
df2_ = df2.loc[start_time:end_time]
df3_ = df3.loc[start_time:end_time]
df4_ = df4.loc[start_time:end_time]

#df1.index

# Print Headers
#print(df1.columns.values)


# Resize figure
custom_figsize=(15,5)

# Change lineweights
lineweights = 2

# Variable to plot
variable = 'Solar Radiation'

# Plot two dataframes into one plot
ax = df1_.plot(y=[variable],lw = lineweights, figsize=custom_figsize,color='blue')
df3_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='yellow',ax=ax)
df2_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='red',ax=ax)
df4_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='magenta',ax=ax)

# Subset Time Series By Dates Python Using Pandas
#https://www.earthdatascience.org/courses/earth-analytics-python/use-time-series-data-in-python/subset-time-series-data-python/


# Legend
ax.legend([
"DL1 - Stone Garden - Tjaden Hall", 
"DL2 - PSB Parking Lot",
"DL3 - Olin Library",
"DL4 - Milstein Hall Roof "  
],loc='upper left', frameon=False)


# Fix xticks

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
days = mdates.DayLocator()  # every minute
hours = mdates.HourLocator()  # every minute
minutes = mdates.MinuteLocator()  # every minute

days_fmt = mdates.DateFormatter('%d')
hours_fmt = mdates.DateFormatter('%H:00')

# format the ticks
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_major_formatter(hours_fmt)
ax.xaxis.set_minor_locator(hours)

#Labels
plt.ylabel('Global Radiation $[W/m^2]$')
plt.xlabel("Time")
     
# Title
ax.set_title(variable)

# Save PDF
plot_graph(plt, start_time, variable)

In [ ]:
from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path
     
    
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   
  
    
S1=HoboReader(loadPath / "20191101_11-04_DL1.csv") 
S2=HoboReader(loadPath / "20191101-10-50_DL2.csv")
S3=HoboReader(loadPath / "20191115-11-08_DL3.csv")
S4=HoboReader(loadPath / "20191101-12-58_DL4.csv")


df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()

# Select time

start_time = '2019-10-17 06:00:00-04:00'
end_time = '2019-10-17 19:00:00-04:00'

df1_ = df1.loc[start_time:end_time]
df2_ = df2.loc[start_time:end_time]
df3_ = df3.loc[start_time:end_time]
df4_ = df4.loc[start_time:end_time]

#df1.index

# Print Headers
#print(df1.columns.values)


# Resize figure
custom_figsize=(15,5)

# Change lineweights
lineweights = 2

# Variable to plot
variable = "Wind Speed"

# Plot two dataframes into one plot
ax = df1_.plot(y=[variable],lw = lineweights, figsize=custom_figsize,color='blue')
df3_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='yellow',ax=ax)
df2_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='red',ax=ax)
df4_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='magenta',ax=ax)

# Subset Time Series By Dates Python Using Pandas
#https://www.earthdatascience.org/courses/earth-analytics-python/use-time-series-data-in-python/subset-time-series-data-python/


# Legend
ax.legend([
"DL1 - Stone Garden - Tjaden Hall", 
"DL2 - PSB Parking Lot",
"DL3 - Olin Library",
"DL4 - Milstein Hall Roof "  
],loc='upper left', frameon=False)


# Fix xticks

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
days = mdates.DayLocator()  # every minute
hours = mdates.HourLocator()  # every minute
minutes = mdates.MinuteLocator()  # every minute

days_fmt = mdates.DateFormatter('%d')
hours_fmt = mdates.DateFormatter('%H:00')

# format the ticks
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_major_formatter(hours_fmt)
ax.xaxis.set_minor_locator(hours)

#Labels
plt.ylabel('Wind velocity $[m/s]$')
plt.xlabel("Time")
     
# Title
ax.set_title(variable)

# Save PDF
plot_graph(plt, start_time, variable)

In [ ]:
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   


#"#","Date Time, GMT-04:00",
#"Temp, °F (LGR S/N: 20702490, SEN S/N: 20496098, LBL: MRT)",
#"Temp, °F (LGR S/N: 20702490, SEN S/N: 20698454, LBL: Drybulb)",
#"RH, % (LGR S/N: 20702490, SEN S/N: 20698454)",
#"Solar Radiation, W/m² (LGR S/N: 20702490, SEN S/N: 20702165, LBL: Rad)",
#"Wind Speed, mph (LGR S/N: 20702490, SEN S/N: 20706788, LBL: Wind)",
#"Gust Speed, mph (LGR S/N: 20702490, SEN S/N: 20706788)",
#"Wind Direction, ø (LGR S/N: 20702490, SEN S/N: 20706788)",
#"DewPt, °F (LGR S/N: 20702490, SEN S/N: 20698454)"
    

S4=HoboReader(loadPath / "20191101-12-58_DL4.csv")


df4=S4.get_dataframe()

# Select time

start_time = '2019-09-01 06:00:00-04:00'
end_time = '2019-12-15 19:00:00-04:00'


df4_ = df4.loc[start_time:end_time]

# Resize figure
custom_figsize=(25,5)

# Change lineweights
lineweights = 2


# Variable to plot
variable = "Temp"
variable2 = "Temp"


test= df4_[variable].subtract(df4_[variable2], fill_value=0)


print(test)


# Plot two dataframes into one plot
#ax = plt.plot(diff,lw = lineweights, figsize=custom_figsize,color='blue')
#
## Subset Time Series By Dates Python Using Pandas
##https://www.earthdatascience.org/courses/earth-analytics-python/use-time-series-data-in-python/subset-time-series-data-python/
#
## Legend
#ax.legend([
#"DL1 - Stone Garden - Tjaden Hall", 
#"DL2 - PSB Parking Lot",
#"DL3 - Olin Library",
#"DL4 - Milstein Hall Roof "  
#],loc='upper left', frameon=False)
#
## Fix xticks
#
#years = mdates.YearLocator()   # every year
#months = mdates.MonthLocator()  # every month
#days = mdates.DayLocator()  # every minute
#hours = mdates.HourLocator()  # every minute
#minutes = mdates.MinuteLocator()  # every minute
#
#days_fmt = mdates.DateFormatter('%d')
#hours_fmt = mdates.DateFormatter('%H:00')
#
## format the ticks
#ax.xaxis.set_major_locator(hours)
#ax.xaxis.set_major_formatter(hours_fmt)
#ax.xaxis.set_minor_locator(hours)
#
##Labels
#plt.ylabel('Wind velocity $[m/s]$')
#plt.xlabel("Time")
#     
## Title
#ax.set_title(variable)

In [ ]:
from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path

    
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   
  
    
S1=HoboReader(loadPath / "20191101_11-04_DL1.csv") 
S2=HoboReader(loadPath / "20191101-10-50_DL2.csv")
S3=HoboReader(loadPath / "20191115-11-08_DL3.csv")
S4=HoboReader(loadPath / "20191101-12-58_DL4.csv")


df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()

# Select time

start_time = '2019-10-17 06:00:00-04:00'
end_time = '2019-10-17 19:00:00-04:00'

df1_ = df1.loc[start_time:end_time]
df2_ = df2.loc[start_time:end_time]
df3_ = df3.loc[start_time:end_time]
df4_ = df4.loc[start_time:end_time]

#df1.index

# Print Headers
#print(df1.columns.values)


# Resize figure
custom_figsize=(15,5)

# Change lineweights
lineweights = 2

# Variable to plot
variable = "Wind Direction"

# Plot two dataframes into one plot
ax = df1_.plot(y=[variable],lw = lineweights, figsize=custom_figsize,color='blue')
df3_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='yellow',ax=ax)
df2_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='red',ax=ax)
df4_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='magenta',ax=ax)

# Subset Time Series By Dates Python Using Pandas
#https://www.earthdatascience.org/courses/earth-analytics-python/use-time-series-data-in-python/subset-time-series-data-python/


# Legend
ax.legend([
"DL1 - Stone Garden - Tjaden Hall", 
"DL2 - PSB Parking Lot",
"DL3 - Olin Library",
"DL4 - Milstein Hall Roof "  
],loc='upper left', frameon=False)


# Fix xticks

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
days = mdates.DayLocator()  # every minute
hours = mdates.HourLocator()  # every minute
minutes = mdates.MinuteLocator()  # every minute

days_fmt = mdates.DateFormatter('%d')
hours_fmt = mdates.DateFormatter('%H:00')

# format the ticks
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_major_formatter(hours_fmt)
ax.xaxis.set_minor_locator(hours)

#Labels
plt.ylabel('Wind direction $[°]$')
plt.xlabel("Time")
     
# Title
ax.set_title(variable)

# Save PDF
plot_graph(plt, start_time, variable)

In [ ]:
# conda install -c https://conda.anaconda.org/ioos geopandas
# conda install -c conda-forge shapely
# conda install -c conda-forge descartes

import geopandas as gpd
from shapely.geometry import Point, Polygon
import descartes

#final_crs = {'init': 'epsg:28992'}

buildings = gpd.read_file('Ithaca/map/buildings-polygon.shp')#.to_crs(final_crs)
roads = gpd.read_file('Ithaca/map/roads-line.shp')#.to_crs(final_crs)
landcover = gpd.read_file('Ithaca/map/landcover-polygon.shp')#.to_crs(final_crs)
water = gpd.read_file('Ithaca/map/water_areas-polygon.shp')#.to_crs(final_crs)
waterlines = gpd.read_file('Ithaca/map/water_lines-line.shp')#.to_crs(final_crs)

fig,ax = plt.subplots(figsize= (10,10))

roads.plot(ax=ax, color= "grey")
landcover.plot(ax=ax, color= "green", alpha=0.03)
water.plot(ax=ax, color= "blue", alpha=0.1)
waterlines.plot(ax=ax, color= "blue", alpha=0.1)
buildings.plot(ax=ax, color= "black")

minx, miny, maxx, maxy = -76.4900, 42.443484, -76.475089, 42.4555
ax.set_xlim(minx, maxx)
ax.set_ylim(miny, maxy)

DL1= Point((-76.485089,42.450960))
DL2= Point((-76.481194,42.450536))
DL3= Point((-76.484268,42.448039))
DL4= Point((-76.483743,42.451245))

gdf_stations = gpd.GeoSeries([DL1,DL2,DL3,DL4])
gdf_stations.plot(ax=ax, markersize= 50, color= "red", marker = "o")

# Annotate with names

stations = [
"DL1", 
"DL2",
"DL3",
"DL4"  
]
xx = 0.0003
yy = 0.0001

for i, geo in gdf_stations.centroid.iteritems():
    ax.annotate(s=stations[i], xy=[geo.x+xx, geo.y-yy], color="red" )


# Save PDF
plt.savefig('Map.pdf')

In [ ]:
from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path


    
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   
  
    
S1=HoboReader(basePathPatrick / "20191101_11-04_DL1.csv") 
S2=HoboReader(basePathPatrick / "20191101-10-50_DL2.csv")
S3=HoboReader(basePathPatrick / "20191115-11-08_DL3.csv")
S4=HoboReader(basePathPatrick / "20191101-12-58_DL4.csv")


df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()

# Select time

start_time = '2019-10-31 00:00:00-04:00'
end_time = '2019-11-01 23:59:00-04:00'

df1_ = df1.loc[start_time:end_time]
df2_ = df2.loc[start_time:end_time]
df3_ = df3.loc[start_time:end_time]
df4_ = df4.loc[start_time:end_time]

#df1.index

# Print Headers
#print(df1.columns.values)


# Resize figure
custom_figsize=(40,15)

# Change lineweights
lineweights = 1.5

# Variable to plot
variable = "Wind Direction"

# Plot two dataframes into one plot
ax = df1_.plot(y=[variable],lw = lineweights, figsize=custom_figsize,color='red')
df3_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='blue',ax=ax)
df2_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='green',ax=ax)
df4_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='aqua',ax=ax)

# Subset Time Series By Dates Python Using Pandas
#https://www.earthdatascience.org/courses/earth-analytics-python/use-time-series-data-in-python/subset-time-series-data-python/


# Legend
ax.legend([
"DL1 - Stone Garden - Tjaden Hall", 
"DL2 - PSB Parking Lot",
"DL3 - Olin Library",
"DL4 - Milstein Hall Roof "  
],loc='upper left', frameon=False)


# Fix xticks

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
days = mdates.DayLocator()  # every minute
hours = mdates.HourLocator()  # every minute
minutes = mdates.MinuteLocator()  # every minute

days_fmt = mdates.DateFormatter('%d')
hours_fmt = mdates.DateFormatter('%H:00')

# format the ticks
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_major_formatter(hours_fmt)
ax.xaxis.set_minor_locator(hours)

#Labels
plt.ylabel('Wind direction $[°]$', fontname="Consolas", fontsize=20)
plt.xlabel("Time", fontname="Consolas", fontsize=20)
     
# Title
ax.set_title(variable, fontname="Consolas", fontsize=28)

# Save PDF
plot_graph(plt, start_time, variable)

In [ ]:
from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path


      
    
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   
  
    
S1=HoboReader(basePathPatrick/ "20191101_11-04_DL1.csv") 
S2=HoboReader(basePathPatrick/ "20191101-10-50_DL2.csv")
S3=HoboReader(basePathPatrick/ "20191115-11-08_DL3.csv")
S4=HoboReader(basePathPatrick/ "20191101-12-58_DL4.csv")


df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()

# Select time

start_time = '2019-10-31 00:00:00-04:00'
end_time = '2019-11-01 23:59:00-04:00'

df1_ = df1.loc[start_time:end_time]
df2_ = df2.loc[start_time:end_time]
df3_ = df3.loc[start_time:end_time]
df4_ = df4.loc[start_time:end_time]

#df1.index

# Print Headers
#print(df1.columns.values)


# Resize figure
custom_figsize=(40,15)

# Change lineweights
lineweights = 1.5

# Variable to plot
variable = "Wind Speed"

# Plot two dataframes into one plot
ax = df1_.plot(y=[variable],lw = lineweights, figsize=custom_figsize,color='red')
df3_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='blue',ax=ax)
df2_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='green',ax=ax)
df4_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='aqua',ax=ax)

# Subset Time Series By Dates Python Using Pandas
#https://www.earthdatascience.org/courses/earth-analytics-python/use-time-series-data-in-python/subset-time-series-data-python/


# Legend
ax.legend([
"DL1 - Stone Garden - Tjaden Hall", 
"DL2 - PSB Parking Lot",
"DL3 - Olin Library",
"DL4 - Milstein Hall Roof "  
],loc='upper left', frameon=False)


# Fix xticks

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
days = mdates.DayLocator()  # every minute
hours = mdates.HourLocator()  # every minute
minutes = mdates.MinuteLocator()  # every minute

days_fmt = mdates.DateFormatter('%d')
hours_fmt = mdates.DateFormatter('%H:00')

# format the ticks
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_major_formatter(hours_fmt)
ax.xaxis.set_minor_locator(hours)

#Labels
plt.ylabel('Wind Speed $[MPH]$', fontname="Consolas", fontsize=20)
plt.xlabel("Time", fontname="Consolas", fontsize=20)
     
# Title
ax.set_title(variable, fontname="Consolas", fontsize=28)

# Save PDF
plot_graph(plt, start_time, variable)

In [ ]:
from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path


    
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   
  
    
S1=HoboReader(basePathPatrick / "20191101_11-04_DL1.csv") 
S2=HoboReader(basePathPatrick / "20191101-10-50_DL2.csv")
S3=HoboReader(basePathPatrick / "20191115-11-08_DL3.csv")
S4=HoboReader(basePathPatrick / "20191101-12-58_DL4.csv")


df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()

# Select time

start_time = '2019-10-31 00:00:00-04:00'
end_time = '2019-11-01 23:59:00-04:00'

df1_ = df1.loc[start_time:end_time]
df2_ = df2.loc[start_time:end_time]
df3_ = df3.loc[start_time:end_time]
df4_ = df4.loc[start_time:end_time]

#df1.index

# Print Headers
#print(df1.columns.values)


# Resize figure
custom_figsize=(40,15)

# Change lineweights
lineweights = 2

# Variable to plot
variable = 'Solar Radiation'

# Plot two dataframes into one plot
ax = df1_.plot(y=[variable],lw = lineweights, figsize=custom_figsize,color='red')
df3_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='blue',ax=ax)
df2_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='green',ax=ax)
df4_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='aqua',ax=ax)

# Subset Time Series By Dates Python Using Pandas
#https://www.earthdatascience.org/courses/earth-analytics-python/use-time-series-data-in-python/subset-time-series-data-python/


# Legend
ax.legend([
"DL1 - Stone Garden - Tjaden Hall", 
"DL2 - PSB Parking Lot",
"DL3 - Olin Library",
"DL4 - Milstein Hall Roof "  
],loc='upper left', frameon=False)


# Fix xticks

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
days = mdates.DayLocator()  # every minute
hours = mdates.HourLocator()  # every minute
minutes = mdates.MinuteLocator()  # every minute

days_fmt = mdates.DateFormatter('%d')
hours_fmt = mdates.DateFormatter('%H:00')

# format the ticks
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_major_formatter(hours_fmt)
ax.xaxis.set_minor_locator(hours)

#Labels
plt.ylabel('Global Radiation $[W/m^2]$', fontname="Consolas", fontsize=20)
plt.xlabel("Time", fontname="Consolas", fontsize=20)
     
# Title
ax.set_title(variable, fontname="Consolas", fontsize=28)

# Save PDF
plot_graph(plt, start_time, variable)

In [ ]:
from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path


    
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   
  
    
S1=HoboReader(basePathPatrick / "20191101_11-04_DL1.csv") 
S2=HoboReader(basePathPatrick / "20191101-10-50_DL2.csv")
S3=HoboReader(basePathPatrick / "20191115-11-08_DL3.csv")
S4=HoboReader(basePathPatrick / "20191101-12-58_DL4.csv")


df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()

# Select time

start_time = '2019-10-31 00:00:00-04:00'
end_time = '2019-11-01 23:59:00-04:00'

df1_ = df1.loc[start_time:end_time]
df2_ = df2.loc[start_time:end_time]
df3_ = df3.loc[start_time:end_time]
df4_ = df4.loc[start_time:end_time]

#df1.index

# Print Headers
#print(df1.columns.values)


# Resize figure
custom_figsize=(40,15)

# Change lineweights
lineweights = 1.5

# Variable to plot
variable = "Temp"

# Translate to °C

df1_[variable] = df1_[variable].apply(lambda x: f2c(x), axis=1)
df2_[variable] = df2_[variable].apply(lambda x: f2c(x), axis=1)
df3_[variable] = df3_[variable].apply(lambda x: f2c(x), axis=1)
df4_[variable] = df4_[variable].apply(lambda x: f2c(x), axis=1)


# Plot two dataframes into one plot
ax = df1_.plot(y=[variable],lw = lineweights, figsize=custom_figsize,color='red')
df3_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='blue',ax=ax)
df2_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='green',ax=ax)
df4_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='aqua',ax=ax)

# Subset Time Series By Dates Python Using Pandas
#https://www.earthdatascience.org/courses/earth-analytics-python/use-time-series-data-in-python/subset-time-series-data-python/


# Legend
ax.legend([
"DL1 - Stone Garden - Tjaden Hall", 
"DL2 - PSB Parking Lot",
"DL3 - Olin Library",
"DL4 - Milstein Hall Roof "  
],loc='upper left', frameon=False)


# Fix xticks

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
days = mdates.DayLocator()  # every minute
hours = mdates.HourLocator()  # every minute
minutes = mdates.MinuteLocator()  # every minute

days_fmt = mdates.DateFormatter('%d')
hours_fmt = mdates.DateFormatter('%H:00')

# format the ticks
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_major_formatter(hours_fmt)
ax.xaxis.set_minor_locator(hours)

#Labels
plt.ylabel('Temp $[°C]$', fontname="Consolas", fontsize=20)
plt.xlabel("Time", fontname="Consolas", fontsize=20)
     
# Title
ax.set_title(variable, fontname="Consolas", fontsize=28)

# Save PDF
plot_graph(plt, start_time, variable)

In [ ]:
# conda install -c https://conda.anaconda.org/ioos geopandas
# conda install -c conda-forge shapely
# conda install -c conda-forge descartes

import geopandas as gpd
from shapely.geometry import Point, Polygon
import descartes

#final_crs = {'init': 'epsg:28992'}

buildings = gpd.read_file('Ithaca/map/buildings-polygon.shp')#.to_crs(final_crs)
roads = gpd.read_file('Ithaca/map/roads-line.shp')#.to_crs(final_crs)
landcover = gpd.read_file('Ithaca/map/landcover-polygon.shp')#.to_crs(final_crs)
water = gpd.read_file('Ithaca/map/water_areas-polygon.shp')#.to_crs(final_crs)
waterlines = gpd.read_file('Ithaca/map/water_lines-line.shp')#.to_crs(final_crs)

fig,ax = plt.subplots(figsize= (10,10))

roads.plot(ax=ax, color= "grey")
landcover.plot(ax=ax, color= "green", alpha=0.03)
water.plot(ax=ax, color= "blue", alpha=0.1)
waterlines.plot(ax=ax, color= "blue", alpha=0.1)
buildings.plot(ax=ax, color= "black")

minx, miny, maxx, maxy = -76.4900, 42.443484, -76.475089, 42.4555
ax.set_xlim(minx, maxx)
ax.set_ylim(miny, maxy)

DL1= Point((-76.485089,42.450960))
DL2= Point((-76.481194,42.450536))
DL3= Point((-76.484268,42.448039))
DL4= Point((-76.483743,42.451245))

gdf_stations = gpd.GeoSeries([DL1,DL2,DL3,DL4])
gdf_stations.plot(ax=ax, markersize= 50, color= "red", marker = "o")

# Annotate with names

stations = [
"DL1", 
"DL2",
"DL3",
"DL4"  
]
xx = 0.0003
yy = 0.0001

for i, geo in gdf_stations.centroid.iteritems():
    ax.annotate(s=stations[i], xy=[geo.x+xx, geo.y-yy], color="red" )


# Save PDF
plt.savefig('Map.pdf')

In [6]:
# Merge all recorded data



from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
%matplotlib inline

pd.set_option('mode.chained_assignment',None)

# Set up figure size and DPI for screen demo
plt.rcParams['figure.figsize'] = (6,4)
plt.rcParams['figure.dpi'] = 150





#loadPath = Path(r"C:\Users\remym\Dropbox\ESL\CSV Merged Data")
#loadPath = Path(r"C:\Users\pkastner\Google Drive\Patrick\OutdoorComfort-Biking\Measurements")

#S1=HoboReader(loadPath / "DL1_Stone_Garden_Tjaden_Hall.csv") 
#S2=HoboReader(loadPath / "DL2_PSB_Parking_Lot.csv")
#S3=HoboReader(loadPath / "DL3_Olin_Library.csv")
#S4=HoboReader(loadPath / "DL4_Milstein_Hall_Roof.csv")
#S5=HoboReader(LoadPath / "DL5_Game_Farm_Road_DST20192020.csv")

    
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   
#DL5	Game Farm Road   
  
    
S1=HoboReader("DL1_Stone_Garden_Tjaden_Hall.csv") 
S2=HoboReader("DL2_PSB_Parking_Lot.csv")
S3=HoboReader("DL3_Olin_Library.csv")
S4=HoboReader("DL4_Milstein_Hall_Roof.csv")
S5=HoboReader("DL5_Game_Farm_Road_DST20192020.csv")



df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()
df5=S5.get_dataframe()

df5=df5.shift(periods=(-1), fill_value=0 )

def ftoc(f):
    return (f - 32) * 5.0/9.0

def mphtoms(mph):
    return mph * 0.44704

def f(variable_id,d, m,y):
            
    start_time = str(y)+'-'+str(m)+'-'+str(d)+' 07:00:00-00:00'
    end_time =   str(y)+'-'+str(m)+'-'+str(d)+' 23:00:00-00:00'
    
   
    df1_ = df1.loc[start_time:end_time]
    df2_ = df2.loc[start_time:end_time]
    df3_ = df3.loc[start_time:end_time]
    df4_ = df4.loc[start_time:end_time]
    df5_ = df5.loc[start_time:end_time]


    # Translate to °C

    df1_["Temp"] = df1_["Temp"].apply(lambda x: ftoc(x), axis=1)
    df2_["Temp"] = df2_["Temp"].apply(lambda x: ftoc(x), axis=1)
    df3_["Temp"] = df3_["Temp"].apply(lambda x: ftoc(x), axis=1)
    df4_["Temp"] = df4_["Temp"].apply(lambda x: ftoc(x), axis=1)
    df5_["Temp"] = df5_["Temp"].apply(lambda x: ftoc(x), axis=1)
    
    df1_["MRT"] = df1_["MRT"].apply(lambda x: ftoc(x), axis=1)
    df2_["MRT"] = df2_["MRT"].apply(lambda x: ftoc(x), axis=1)
    df3_["MRT"] = df3_["MRT"].apply(lambda x: ftoc(x), axis=1)
    df4_["MRT"] = df4_["MRT"].apply(lambda x: ftoc(x), axis=1)
    #df5_["MRT"] = df5_["MRT"].apply(lambda x: ftoc(x), axis=1)
    
    # Translate to m/s
    
    df1_["Wind Speed"] = df1_["Wind Speed"].apply(lambda x: mphtoms(x), axis=1)
    df2_["Wind Speed"] = df2_["Wind Speed"].apply(lambda x: mphtoms(x), axis=1)
    df3_["Wind Speed"] = df3_["Wind Speed"].apply(lambda x: mphtoms(x), axis=1)
    df4_["Wind Speed"] = df4_["Wind Speed"].apply(lambda x: mphtoms(x), axis=1)
    df5_["Wind Speed"] = df5_["Wind Speed"].apply(lambda x: mphtoms(x), axis=1)
    
    df1_["Gust Speed"] = df1_["Gust Speed"].apply(lambda x: mphtoms(x), axis=1)
    df2_["Gust Speed"] = df2_["Gust Speed"].apply(lambda x: mphtoms(x), axis=1)
    df3_["Gust Speed"] = df3_["Gust Speed"].apply(lambda x: mphtoms(x), axis=1)
    df4_["Gust Speed"] = df4_["Gust Speed"].apply(lambda x: mphtoms(x), axis=1)
    #df5_["Gust Speed"] = df5_["Gust Speed"].apply(lambda x: mphtoms(x), axis=1)
    
    
    #df1.index
    
    # Print Headers
    #print(df1.columns.values)
    
    
    # Resize figure
    custom_figsize=(20,10)
    
    # Change lineweights
    lineweights = 1.5
    
    

    
    variables = ["Wind Speed","MRT","Temp","RH","Solar Radiation","Wind Speed","Gust Speed","Wind Direction","DewPt" ]  
    
    
    
    variable = variables[variable_id]
    
    # Plot both temp and mrt when temp is selected
    
    if variable == "MRT" or variable == "DewPt" or variable == "Gust Speed" :
        ax = df1_.plot(y = variable, lw=1, figsize=custom_figsize,color='red')
        df2_.plot( y = variable, lw=1, figsize=custom_figsize,color='blue',ax=ax)
        df3_.plot(y = variable, lw=1, figsize=custom_figsize,color='green',ax=ax)
        df4_.plot(y = variable, lw=1, figsize=custom_figsize,color='aqua',ax=ax)  
    
    elif variable == "Temp":
        # Plot two dataframes into one plot
        ax = df1_.plot(y = variable, lw=2, figsize=custom_figsize,color='red')
        df2_.plot( y = variable,    lw=2, figsize=custom_figsize,color='blue',ax=ax)
        df3_.plot(y = variable,     lw=2, figsize=custom_figsize,color='green',ax=ax)
        df4_.plot(y = variable,     lw=2, figsize=custom_figsize,color='aqua',ax=ax)
        df5_.plot(y = variable,     lw=3.5, figsize=custom_figsize,color='black',ax=ax)
        
        ax = df1_.plot(y = "MRT", lw=1, figsize=custom_figsize,style = ['--'],color='red', ax=ax)
        df2_.plot( y = "MRT",     lw=1, figsize=custom_figsize, style = ['--'],color='blue',ax=ax)
        df3_.plot(y = "MRT",      lw=1, figsize=custom_figsize, style = ['--'],color='green',ax=ax)
        df4_.plot(y = "MRT",      lw=1, figsize=custom_figsize, style = ['--'],color='aqua',ax=ax)      
                  
    else:  
        # Plot two dataframes into one plot
        ax = df1_.plot(y = variable, lw=1, figsize=custom_figsize,color='red')
        df2_.plot( y = variable, lw=1, figsize=custom_figsize,color='blue',ax=ax)
        df3_.plot(y = variable, lw=1, figsize=custom_figsize,color='green',ax=ax)
        df4_.plot(y = variable, lw=1, figsize=custom_figsize,color='aqua',ax=ax)
        df5_.plot(y = variable, lw=2, figsize=custom_figsize,color='black',ax=ax)
    
    # Legend
    ax.legend([
    "DL1 - Stone Garden - Tjaden Hall", 
    "DL2 - PSB Parking Lot",
    "DL3 - Olin Library",
    "DL4 - Milstein Hall Roof ",
    "DL5 - Game Farm Road"
    
    ],loc='upper left', frameon=False)
    
    
    # Fix xticks
    
    years = mdates.YearLocator()   # every year
    months = mdates.MonthLocator()  # every month
    days = mdates.DayLocator()  # every minute
    hours = mdates.HourLocator()  # every minute
    minutes = mdates.MinuteLocator()  # every minute
    
    days_fmt = mdates.DateFormatter('%d')
    hours_fmt = mdates.DateFormatter('%H:00')
    
    # format the ticks
    ax.xaxis.set_major_locator(hours)
    ax.xaxis.set_major_formatter(hours_fmt)
    ax.xaxis.set_minor_locator(hours)
    
    #Labels
    plt.ylabel(variable, fontname="Consolas", fontsize=20)
    plt.xlabel("UTC Time", fontname="Consolas", fontsize=20)
         
    # Title
    ax.set_title(str(start_time.split(' ')[0]+" "+variable), fontname="Consolas", fontsize=28)
    
    # Save PDF
    #plot_graph(plt, start_time, variable)

    
# Variable to plot
#1 = "MRT °C"
#2 = "Temp °C"
#3 = "RH"
#4 = "Solar Radiation"
#5 = "Wind Speed"
#6 = "Gust Speed"
#7 = "Wind Direction"
#8 = "DewPt"
    
    
interact(f,variable_id=(1,8,1), d=(1,31,1), m=(1,12,1), y = (2019,2020,1))










interactive(children=(IntSlider(value=4, description='variable_id', max=8, min=1), IntSlider(value=16, descrip…

<function __main__.f(variable_id, d, m, y)>

In [87]:
display(df5)

title,#,Date Time,Temp,RH,Wind Speed,Wind Direction,Solar Radiation
timezone_str,NaN,GMT-05:00,NaN,NaN,NaN,NaN,NaN
units,NaN,NaN,Â°F,%,mph,Ã¸,W/mÂ²
logger_serial_number,NaN,NaN,20098520,21098500,20198500,20198500,20034090
sensor_serial_number,NaN,NaN,10762331,10263331,21706789,21706789,23073988
datetimes,,,,,,,
2019-05-27 00:00:00-05:00,2,05/27/19 01:00:00 AM,54.37,93.70,3.32,116.7,0.00
2019-05-27 01:00:00-05:00,3,05/27/19 02:00:00 AM,52.05,94.70,3.16,111.7,0.00
2019-05-27 02:00:00-05:00,4,05/27/19 03:00:00 AM,49.89,96.20,3.50,121.4,0.00
2019-05-27 03:00:00-05:00,5,05/27/19 04:00:00 AM,49.01,96.70,2.90,109.8,0.00
2019-05-27 04:00:00-05:00,6,05/27/19 05:00:00 AM,47.77,97.30,3.28,95.8,0.00


# Direct/Diffuse split


We need the extra terrestrial radiation for the diffuse part which we can get with "get_extra_radiation()".

https://pvlib-python.readthedocs.io/en/stable/generated/pvlib.irradiance.get_extra_radiation.html?highlight=dni_extra%20

From there, we can use "reindl()" to calculate the split.

https://pvlib-python.readthedocs.io/en/stable/generated/pvlib.irradiance.reindl.html?highlight=reindl


https://pvpmc.sandia.gov/modeling-steps/1-weather-design-inputs/plane-of-array-poa-irradiance/

In [261]:
S5 = HoboReader("DL5_Game_Farm_Road_last365v2.csv")

df5 = S5.get_dataframe()


#!pip install pvlib
import pvlib
# https://pvlib-python.readthedocs.io/en/stable/generated/pvlib.irradiance.reindl.html?highlight=reindl#pvlib.irradiance.reindl

# conda install -c pvlib pvlib
import pandas as pd

# Create date series using date_range() function
date_series = pd.date_range('01/01/2019', periods = 8760, freq ='H')
#print(date_series)

# altitude GFR Station
#altitude = 294

# lat, lon taken from GFR
lon = 42.449 # north-positive
lat =  76.449 # west-positive

pos = pvlib.solarposition.get_solarposition(date_series,lat, lon)
pos

import math

solar_constant_e = 1367.0
DTR = 0.017453292  # Pi / 180
RTD = 57.2957795  # 180 / Pi


time_zone = 5*15 # west-positive


def month_and_day_to_julian_day(month, day):
    if (month == 1): return day
    if (month == 2): return day + 31
    if (month == 3): return day + 59
    if (month == 4): return day + 90
    if (month == 5): return day + 120
    if (month == 6): return day + 151
    if (month == 7): return day + 181
    if (month == 8): return day + 212
    if (month == 9): return day + 243
    if (month == 10): return day + 273
    if (month == 11): return day + 304
    if (month == 12):
        return day + 334
    else:
        print("bad month")


def solar_elev_azi_ecc(latitude,
                       longitude,
                       time_zone,
                       jday,
                       time,
                       solar_time=0):
    # /*  angles in degrees, times in hours  */

    # /*  solar elevation and azimuth formulae from sun.c  */
    if (solar_time == 1): sol_time = time
    if (solar_time == 0):
        sol_time = time + 0.170 * math.sin(
            (4 * math.pi / 373) * (jday - 80)) - 0.129 * math.sin(
                (2 * math.pi / 355) *
                (jday - 8)) + 12 / 180.0 * (time_zone - longitude)

    solar_declination = RTD * 0.4093 * math.sin(
        (2 * math.pi / 368) * (jday - 81))
    jday_angle = 2 * math.pi * (jday - 1) / 365

    solar_elevation = RTD * math.asin(
        math.sin(latitude * DTR) * math.sin(solar_declination * DTR) -
        math.cos(latitude * DTR) * math.cos(solar_declination * DTR) *
        math.cos(sol_time * (math.pi / 12)))

    solar_azimuth = RTD * (-math.atan2(
        math.cos(solar_declination * DTR) * math.sin(sol_time *
                                                     (math.pi / 12)),
        -math.cos(latitude * DTR) * math.sin(solar_declination * DTR) -
        math.sin(latitude * DTR) * math.cos(solar_declination * DTR) *
        math.cos(sol_time * (math.pi / 12))))

    # /*  eccentricity_correction formula used in genjdaylit.c */

    eccentricity_correction = 1.00011 + 0.034221 * math.cos(
        jday_angle) + 0.00128 * math.sin(jday_angle) + 0.000719 * math.cos(
            2 * jday_angle) + 0.000077 * math.sin(2 * jday_angle)

    return solar_elevation, solar_azimuth, eccentricity_correction


def diffuse_fraction(irrad_glo, solar_elevation, eccentricity_correction):
    dif_frac = 0

    if (solar_elevation > 0):
        irrad_ex = solar_constant_e * eccentricity_correction * math.sin(
            DTR * solar_elevation)
    else:
        irrad_ex = 0

    if (irrad_ex > 0):
        index_glo_ex = irrad_glo / irrad_ex
    else:
        return 0

    if (index_glo_ex < 0): print("negative irrad_glo in diffuse_fraction_th\n")
    if (index_glo_ex > 1): index_glo_ex = 1

    if (index_glo_ex <= 0.3):
        dif_frac = 1.02 - 0.254 * index_glo_ex + 0.0123 * math.sin(
            DTR * solar_elevation)
    if (dif_frac > 1):
        dif_frac = 1

    if (0.3 < index_glo_ex < 0.78):
        dif_frac = 1.4 - 1.749 * index_glo_ex + 0.177 * math.sin(
            DTR * solar_elevation)
    if (dif_frac > 0.97):
        dif_frac = 0.97
    if (dif_frac < 0.1): dif_frac = 0.1

    if (index_glo_ex >= 0.78):
        dif_frac = 0.486 * index_glo_ex - 0.182 * math.sin(
            DTR * solar_elevation)
    if dif_frac < 0.1:
        dif_frac = 0.1

    return dif_frac


def calc_split(month, day, time, irrad_glo):
    jday = month_and_day_to_julian_day(month, day)

    if (irrad_glo < 0 or irrad_glo > solar_constant_e
        ):  # / * check irradiances and exit if necessary * /
        irrad_glo = solar_constant_e

    # no object orientation yet

    solar_elevation, solar_azimuth, eccentricity_correction = solar_elev_azi_ecc(
        lat, lon, time_zone, jday, time)

    #

    irrad_dif = diffuse_fraction(irrad_glo, solar_elevation,
                                 eccentricity_correction) * irrad_glo

    if (solar_elevation > 5.0):

        irrad_beam_nor = (irrad_glo - irrad_dif) * 1.0 / math.sin(
            DTR * solar_elevation)

    else:
        irrad_beam_nor = 0
        irrad_dif = irrad_glo

    if (irrad_beam_nor > solar_constant_e):
        irrad_beam_nor = solar_constant_e
        irrad_dif = irrad_glo - irrad_beam_nor * math.sin(
            DTR * solar_elevation)

    #print(month, day, round(time, 2), round(irrad_beam_nor, 2), round(irrad_dif, 2))

    return irrad_beam_nor, irrad_dif


#print("month, day, time, GHR\n")
calc_split(4, 22, 8.333333333, 107)



def calc_split_df(df):
    df['hour'] = df.index.hour
    df['day'] = df.index.day
    df['month'] = df.index.month
    df['DHR'] = 0
    df['DNI'] = 0
    
    i = 0    
    for index, row in df.iterrows():
        #while i <50:
            GHR = df.iloc[i,df.columns.get_loc('Solar Radiation')].values[0]
            DNI, DHR = calc_split(df.month[i], df.day[i], df.hour[i], GHR)
            #print(GHR, DNI, DHR)
            df.iloc[i, df.columns.get_loc('DHR')] = DHR
            df.iloc[i, df.columns.get_loc('DNI')] = DNI
            i+=1
            
    return df


df5 = calc_split_df(df5)

0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
4.49 0.28587781161960196 4.3553
71.81 4.1883647147020415 69.6557
217.67 19.65020354933584 206.9329554095128
359.04 212.85665865311987 238.74154535314926
531.83 679.2429505281236 145.11357704241874
673.2 797.1544707930477 227.87568875030803
788.77 903.0482400322564 306.7098927603917
869.55 1058.3803666391536 344.01947319599094
865.06 1146.9256316724463 349.6705551015478
808.96 1198.5696375396824 334.9246844672072
704.62 1197.8097419501198 299.0207354732359
553.15 1106.7838272791405 240.38139939946745
393.82 947.0446249736372 174.8220866696955
223.28 713.0168471012519 89.18268182372877
63.95 45.57936125990446 56.83893587685993
2.24 0.48957463227454934 2.1728
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
7.85 0.45572074310293686 7.6145
20.2 1.104215163054958 19.593999999999998
41.51 2.1941115158205378 40.2647
74.05 3.8853647632379165 71.82849999999999
87.52

659.74 783.1491437955816 204.99125064485517
796.62 880.8188371444623 306.4789586085023
845.99 990.3842166811048 331.10439926037594
859.45 1089.5163921276649 343.6385874047338
821.3 1153.9531121387574 336.33173714182215
691.15 1101.7168515079948 290.1098584628691
537.44 993.2399853138905 231.01962083388065
378.11 823.4917741375424 166.03955368157693
213.18 866.9611727630457 27.175209947270023
96.49 140.11667820331868 70.918549193064
5.61 1.0289224891698252 5.4417
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
11.22 0.6818202800710992 10.8834
85.27 4.766755898873529 82.7119
227.77 12.014987080248929 220.9369
396.07 259.5422818254223 243.54351452289592
552.02 675.5914609424715 151.90586303485574
682.18 792.6135952341853 220.79413099514377
814.57 898.6465917863405 313.1627422078987
883.01 1031.1150167639387 345.34300297880776
816.82 1032.4577318196245 326.3533416598471
810.08 1134.2691679624093 331.4909206072284
714.71 1134.5450414619277 299

481.34 745.2711021257389 158.5610662024044
342.21 664.6342100513567 91.51629158313462
414.02 753.8783134440757 171.36075642877924
262.55 807.2120504102278 43.84073415408765
269.28 717.782314459316 105.73312208348231
99.86 122.82065201035657 75.84405763370113
12.34 2.1009569973569233 11.9698
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
5.61 0.3357162093232032 5.4417
72.93 4.018660886838841 70.74210000000001
141.37 7.353003225355252 137.1289
233.38 18.72521774307182 222.21676237430984
353.43 159.16950349783377 257.7305538253277
531.83 614.5029184433313 168.23871574874764
753.98 819.3601628605047 288.47221852378317
476.85 618.6110155533083 147.62302276705108
310.79 229.30179627994917 199.2723369852922
532.95 771.0796689054927 198.56499993271308
422.99 736.088362102569 144.9247595211072
345.58 712.6436023088481 115.78621485817263
290.6 702.3420354082895 99.91196239658088
112.2 102.73094159938779 88.73786746181878
14.59 2.2327686440112453 1

191.86 570.538919815411 62.1085498356809
84.15 59.57594227648758 72.55516025429124
8.98 1.5416167020687637 8.7106
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
2.24 0.1352795027761788 2.1728
67.32 3.7380422118432497 65.3004
207.57 15.415185101152078 198.73508154796954
243.47 15.759635666800929 234.12367064009305
403.92 261.07826048839917 247.61318662353676
629.44 762.9443047645346 179.55226324902478
701.25 801.92760936257 246.8796466630997
595.78 767.4287770982911 188.2418973591359
731.54 907.7294112081823 290.8976798142083
500.41 755.605092460822 173.32669499388547
371.38 824.873122319649 60.44196469612576
465.63 829.2327335142458 199.05294545540644
422.99 881.5699246423037 184.77031221348994
292.84 731.3736619098644 127.04592667760579
122.3 252.84598708608058 73.28819329524036
7.85 1.3540979829955564 7.6145
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
4.49 0.27171477683933354 4.3553
8

98.74 162.55823188224932 69.54032488857636
8.98 1.6925141882910153 8.7106
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
39.27 2.246196693576663 38.0919
133.52 7.179089258462513 129.5144
302.94 99.86122310278338 245.1743048976255
311.92 108.11767274661294 248.7136583738049
216.55 16.984235766228956 206.76747907612386
206.45 16.767671177189815 197.17485204511166
430.85 507.45889791552844 168.1373466350399
296.21 217.70165210161696 193.44832046247413
270.4 244.65655345041753 167.82732807210095
290.6 476.6268683182471 117.60134017368046
317.53 698.6874419894913 103.02991643228235
190.74 406.6371582941155 86.97243575143534
117.81 168.27207801025793 82.29838077743904
94.25 143.84394110274442 68.70767480577295
8.98 1.7152269833677483 8.7106
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
1.12 0.07025237995178175 1.0864
21.32 1.2241481036907704 20.6804
87.52 4.722203095229012 84.894399

...

99.41 258.92716789752654 59.71241584107049
53.18 158.05460360036014 37.94974344194705
25.92 0 25.92
25.47 0 25.47
15.03 0 15.03
8.75 0 8.75
1.12 0 1.12
0.0 0 0.0
0.0 0 0.0
0.0 0 0.0
0.0 0 0.0
0.0 0 0.0
0.0 0 0.0
0.0 0 0.0
0.0 0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.11 0.015230315612411581 0.1067
12.34 1.4647445903920897 11.9698
21.43 2.3393871018538723 20.7871
36.47 3.8881355404150244 35.3759
142.83 116.29662845240432 111.1939690826918
134.86 137.15170317808204 100.93461282746448
120.28 170.74374180006546 84.58440035232233
144.51 656.2576346462878 39.67470066647887
146.42 758.8891252576242 68.42118669983574
170.43 0 170.43
74.16 0 74.16
186.14 0 186.14
53.97 0 53.97
2.47 0 2.47
0.0 0 0.0
0.0 0 0.0
0.0 0 0.0
0.0 0 0.0
0.0 0 0.0
0.0 0 0.0
0.0 0 0.0
0.0 0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
7.74 0.8946381197209398 7.5078
59.58 6.348731158963221 57.7926
62.5 6.509714310854613 60.625
52.17 5.617989582229327 50.6049
69.56 8.220649937513224 67.4732
216.1 843.8742913831878 34.244253872258284
152.1

0.0 0 0.0
0.0 0 0.0
0.0 0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.11 0.015366606252405868 0.1067
0.34 0.03783335560618582 0.32980000000000004
2.13 0.2022197790164423 2.0661
17.84 1.525836539255865 17.3048
27.94 2.2560707884212317 27.1018
38.82 3.0899879699705806 37.6554
128.92 14.501868442709545 123.60529425608114
187.71 136.45915684634576 141.17620536534503
262.77 578.4781768369777 87.95099247069317
437.8 970.1197575696025 192.63442531844993
114.78 186.97164493088667 78.14909752766461
85.5 243.76811110790743 52.426504581179906
47.35 0 47.35
42.64 0 42.64
116.58 0 116.58
17.62 0 17.62
0.0 0 0.0
0.0 0 0.0
0.0 0 0.0
0.0 0 0.0
0.0 0 0.0
0.0 0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
7.07 0.6582404531859875 6.8579
46.11 3.875899217082077 44.7267
198.59 110.59948227606382 156.8292193413821
449.92 762.9949413285491 157.78724662238287
583.66 911.7944618200922 244.09915212317273
627.53 1044.0237257696044 265.36019275572147
631.8 1169.0783465719003 271.6293195269363
460.69 999.5135629028392 20

62.61 10.063286916922683 60.501655389188926
41.4 8.226516438438654 40.157999999999994
36.02 17.960012064298066 34.27292607124474
14.47 0 14.47
2.13 0 2.13
0.11 0 0.11
0.11 0 0.11
0.0 0 0.0
0.0 0 0.0
0.0 0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
6.06 0.4578732875494267 5.8782
28.16 1.9618934288714294 27.3152
61.37 4.0885747316380385 59.52889999999999
93.35 6.157540890272759 90.5495
134.64 9.60954302365131 130.37517767062963
83.03 5.958208023067403 80.5391
74.95 5.927785845001913 72.7015
67.66 6.146963539914959 65.63019999999999
127.57 72.16200345778068 107.79055754881024
176.94 519.1633758686321 65.42516632944192
99.3 251.46855224973396 59.98254426708789
68.22 294.0675709025641 38.00652417190643
106.81 0 106.81
48.36 0 48.36
0.22 0 0.22
0.0 0 0.0
0.0 0 0.0
0.0 0 0.0
0.0 0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
41.4 3.0874012837705735 40.157999999999994
172.23 19.413615187649317 163.77047244408823
321.0 305.9290422228042 181.68394119517112

134.19 8.269626007436328 130.1643
243.81 67.13353858538606 209.8491874160594
297.78 158.56904530574005 216.91249536376085
363.53 337.0126440478503 195.40606112326978
383.61 465.068214619881 163.5050489196621
251.1 159.4719678310897 181.73696190319365
323.25 520.8196379422455 121.94464436804225
281.62 553.0829772984796 98.4048568219633
405.27 836.9197369182824 176.8282669501527
123.64 177.8949434497822 85.29096212087289
89.42 157.32656162374684 63.772637934964074
57.22 99.60560986408484 45.378626760668915
27.71 0 27.71
0.79 0 0.79
0.0 0 0.0
0.0 0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.22 0.01593460757030641 0.2134
46.0 3.0114018597380303 44.62
170.32 13.78460449725448 163.55290521838145
357.92 298.9599952277598 205.4675103487285
525.88 863.8798169034116 81.81165964854968
233.49 54.32034015683178 206.17078967391174
252.56 109.05600503824162 200.50047890501548
294.19 265.8909983853234 177.43909096518973
342.55 592.0081839717274 111.24037064974792
557

708.88 1280.404067167558 303.68766268735146
565.82 1221.7221859728318 248.21775475291122
397.41 1053.337627013401 178.08077027185936
234.39 773.8666072848637 106.88488944188897
82.02 239.98627322745892 50.21074153567586
2.58 0.6803432290851247 2.5026
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
1.57 0.10365679424812559 1.5229
69.12 4.165568073001287 67.0464
223.73 25.273868673496125 210.3353031475753
401.12 337.40938908887927 215.96624511602056
563.47 855.1433620621838 90.72695710349763
705.85 812.6703591813562 265.446989929684
815.25 959.2287084184177 319.5329516373149
873.25 1095.7252710857815 348.2498868807025
872.24 1197.682502849189 355.40428178043146
816.14 1260.6205652232989 340.6132689213542
704.73 1260.601933978064 301.46866207552273
556.74 1187.4443510584197 243.8786390158726
390.68 1019.2242686241622 174.8108838981134
228.1 737.7904638868315 103.86617650184797
78.54 191.574755398261 52.44761053783532
2.69 0.6872148240882152 

In [262]:
df5

title,#,Date Time,Temp,RH,Wind Speed,Wind Direction,Solar Radiation,hour,day,month,DHR,DNI
timezone_str,NaN,GMT-05:00,NaN,NaN,NaN,NaN,NaN,,,,,
units,NaN,NaN,Â°F,%,mph,Ã¸,W/mÂ²,,,,,
logger_serial_number,NaN,NaN,20098520,21098500,20198500,20198500,20034090,,,,,
sensor_serial_number,NaN,NaN,10762331,10263331,21706789,21706789,23073988,,,,,
datetimes,,,,,,,,,,,,
2019-05-27 00:00:00-05:00,1,05/27/19 12:00:00 AM,56.53,91.40,3.02,123.6,0.00,0,27,5,0.000000,0.000000
2019-05-27 01:00:00-05:00,2,05/27/19 01:00:00 AM,54.37,93.70,3.32,116.7,0.00,1,27,5,0.000000,0.000000
2019-05-27 02:00:00-05:00,3,05/27/19 02:00:00 AM,52.05,94.70,3.16,111.7,0.00,2,27,5,0.000000,0.000000
2019-05-27 03:00:00-05:00,4,05/27/19 03:00:00 AM,49.89,96.20,3.50,121.4,0.00,3,27,5,0.000000,0.000000
2019-05-27 04:00:00-05:00,5,05/27/19 04:00:00 AM,49.01,96.70,2.90,109.8,0.00,4,27,5,0.000000,0.000000


In [73]:
import pandas as pd
from pandas import read_csv
import csv
import unittest
from pprint import pprint
import pandas as pd
from setuptools import setup, find_packages
from codecs import open
from os import path

loadPath = Path(r"C:\Users\remym\Dropbox\ESL\CSV Merged Data")

read_csv =("DL5_Game_Farm_Road_DST20192020.csv")

# OBJECT CREATION
    
def test_epw___init__(self):
        a=epw()
        self.assertIsInstance(a,epw)
        print(a)


def test_epw_read(self):
        a=epw()
        a.read(DL5_Game_Farm_Road_DST20192020.csv)
        print(a.headers)
        print(a.dataframe.columns)
        print(a.dataframe[['Year', 'Month', 'Day', 'Hour', 'Minute','Dry Bulb Temperature']].head())
        

def test_modify(self):
        a=epw()
        a.read(DL5_Game_Farm_Road_DST20192020.csv)
        
        a.headers['LOCATION'][0]='Ithaca_NY'
        print(a.headers['LOCATION'])
        
        a.dataframe['Dry Bulb Temperature']=old_file['Dry Bulb Temperature']+1.0
        print(a.dataframe[['Year', 'Month', 'Day', 'Hour', 'Minute','Dry Bulb Temperature']].head())

a.write=("new_epw_file")
path= (r"C:\Users\remym\Documents\GitHub\HoboPlot")




(7.791089993154327, 103.78999999999999)

array([0.00380762, 0.00571143, 0.00761524, 0.00951905])